In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('hi?')

['hi', '?']

In [4]:
def stem(word):
    return stemmer.stem(word)

In [5]:
stem('orgnic')

'orgnic'

In [6]:
stem('organic')

'organ'

In [7]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[w] = 1.0
    return bag

In [8]:
bag_of_words(['how'],['are','you','how'])

In [9]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [10]:
bag_of_words(['how'],['are','you','how'])

array([0., 0., 1.], dtype=float32)

In [11]:
data = pd.read_csv('./data.csv')

In [12]:
data

         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this  # earthquake...       1  
1              Forest fire near La Ronge Sask .  Canada       1  
2     All residents asked to  ' shelter in place '  ...       1  
3     13,000 people receive  # wildfires evacuation ...       1  
4     Just got sent this photo from Ruby  # Alaska a...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609   @ Aria Ahrary  @ TheTawniest The out of contr...       1  
7610  M1 . 94  [ 01 :

In [13]:
X = data['text']

In [14]:
y = data['target']

In [15]:
all_words = []
all_data = []
tags = []

In [16]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(Xb)
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

In [17]:
X = []
y = []

In [18]:
np.random.shuffle(all_data)
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [19]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [20]:
np.random.shuffle(all_data)
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [21]:
tags

[0, 1]

In [22]:
X = []
y = []

In [23]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,,all_words))
    y.append(tags.index(tag))

In [24]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tags.index(tag))

In [25]:
X_train,X_test,y_train,y_test = trangeain_test_split(X,y,test_size=0.125,shuffle=False)

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [27]:
from sklearn.model_selection import *

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [29]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).long()

In [30]:
device = 'cuda'

In [31]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).long()

In [32]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [33]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [34]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.iters = 12
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)
        self.output_ac = Sigmoid()
        

In [35]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.iters = 12
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)
        self.output_ac = Sigmoid()
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output_ac(self.output(preds))
        return preds

In [36]:
model = Model()

In [37]:
model = Model(512).to(device)

In [38]:
criterion = MSELoss()

In [39]:
optimizer = Adam(model.parameters(),lr=0.001)

In [40]:
epochs = 100

In [41]:
batch_size = 32

In [42]:
PROJECT_NAME = 'NLP-with-Disaster-Tweets-Clf-V7'

In [43]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()

In [44]:
import wandb

In [45]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

In [46]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()